In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy

from collections import Counter
import plotly.express as pe
from dotenv import load_dotenv
import plotly.io as pio
pio.renderers.default='notebook'

from datetime import date
from datetime import datetime
import copy

import tqdm
from IPython.display import clear_output

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [3]:
from dython import nominal

----------

In [4]:
load_dotenv('.env')

host = os.getenv('EVENTS_DB_HOST')
db = os.getenv('EVENTS_DB_NAME')
user = os.getenv('EVENTS_DB_USER')
password = os.getenv('EVENTS_DB_PASSWORD')
port = os.getenv('EVENTS_DB_PORT')
connection_str = 'postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, password, host, port, db)

engine = sqlalchemy.create_engine(connection_str, execution_options={"stream_results":True})

In [5]:
ACCOUNT_ID = 11

In [6]:
from dateutil import rrule
from datetime import timedelta

In [7]:
%%time
query_sessions = f'''
    select *
    from data.customer_profile_sessions cps 

    left join(
    select id as beh_id,guest_id, customer_profile_id from data.customer_profile_behaviour cpb
    where cpb.account_id = {ACCOUNT_ID}
    ) cpb on cpb.beh_id = cps.customer_profile_behaviour_id


    where cps.account_id = {ACCOUNT_ID} and cps.garbage_session = False and cps.session_start >= '2023-01-01' and cps.session_start < '2023-02-01' 
'''

sessions = pd.read_sql_query(query_sessions, engine)


query_sessions1 = f'''
    select *
    from data.customer_profile_session_journey_statuses
    where account_id = {ACCOUNT_ID} 
    and created >= '2022-12-25' and created < '2023-02-05'

'''
sessions1 = pd.read_sql_query(query_sessions1, engine)

query_sessions2 = f'''
    select *
    from data.customer_journey_statuses
    
'''
statuses = pd.read_sql_query(query_sessions2, engine)

query_sessions3 = f'''
    select *
    from data.sessions_campaign_dict
    
'''
campaigns = pd.read_sql_query(query_sessions3, engine)

query_sessions4 = f'''
    select *
    from data.sessions_source_dict
    
'''
sources = pd.read_sql_query(query_sessions4, engine)

query_sessions5 = f'''
    select *
    from data.sessions_medium_dict
    
'''
mediums = pd.read_sql_query(query_sessions5, engine)

CPU times: user 830 ms, sys: 73.7 ms, total: 903 ms
Wall time: 1.27 s


In [8]:
sessions['duration'] = sessions['session_end'] - sessions['session_start']
sessions['duration'] = sessions['duration'].apply(lambda x: x.total_seconds())

In [9]:
statuses = statuses.rename(columns={'id': 'journey_status_id'})
sources = sources.rename(columns={'id': 'source_id'})
mediums = mediums.rename(columns={'id': 'medium_id'})
campaigns = campaigns.rename(columns={'id': 'campaign_id'})

In [10]:
sessions1 = sessions1.merge(statuses[['journey_status_id', 'status_name']], how='left', on='journey_status_id')

In [11]:
sessions = sessions.merge(sources, how='left', on='source_id').merge(mediums, how='left', on='medium_id').merge(campaigns, how='left', on='campaign_id')

In [12]:
sessions.id.nunique()

25149

In [13]:
sessions = sessions.merge(sessions1[['id', 'journey_status_id', 'created', 'status_name']], how='inner', on='id')
sessions.id.nunique()

25149

In [14]:
chanls = sessions.channel.unique().tolist()
chanls

['Organic Search', 'Social', 'Direct', 'Referral', 'Email', 'Paid Search']

In [15]:
types = sessions.status_name.unique().tolist()
types

['Consideration',
 'Bounce',
 'Awareness',
 'Acquisition',
 'Service',
 'LoyaltyPlus',
 'Loyalty']

------

In [16]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25149 entries, 0 to 25148
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             25149 non-null  int64         
 1   s_id                           25149 non-null  object        
 2   account_id                     25149 non-null  int64         
 3   customer_profile_behaviour_id  25149 non-null  int64         
 4   channel                        25149 non-null  object        
 5   utm_source                     15709 non-null  object        
 6   utm_medium                     15553 non-null  object        
 7   utm_campaign                   15034 non-null  object        
 8   utm_content                    15409 non-null  object        
 9   utm_term                       5 non-null      object        
 10  ip                             25149 non-null  object        
 11  ipcountry      

In [17]:
sliced = sessions[['source', 'medium', 'campaign', 'ipcountry', 'device_family', 'add_to_basket_count', 'duration', 'status_name']]
smcid = [['source_id', 'medium_id', 'campaign_id', 'ipcountry', 'device_family']]
kpiv = ['add_to_basket_count', 'duration', 'status_name']
sliced.head()

,source,medium,campaign,ipcountry,device_family,add_to_basket_count,duration,status_name
0,google.com,referral,(none),GB,Huawei CLT-L29,0,385.043,Consideration
1,meta,Facebook_Mobile_Feed,TOFU_Winter22_Prospecting_Europe_Excluded-LT,IE,Samsung SM-A037G,0,8.683,Bounce
2,meta,Facebook_Marketplace,TOFU_Winter22_Prospecting_Europe_Excluded-LT,NL,iPhone,0,30.002,Bounce
3,IGShopping,Social,(none),NO,iPhone,0,15.805,Bounce
4,(direct),(none),(none),GB,iPhone,0,38.233,Awareness


In [18]:
sliced.groupby(['source', 'medium', 'campaign', 'ipcountry', 'device_family']).size()

source       medium    campaign  ipcountry  device_family   
(direct)     (none)    (none)    AE         Other               1
                                            iPhone              1
                                 AF         Samsung SM-G950F    2
                                 AT         2203121C            1
                                            Huawei VOG-L29      2
                                                               ..
youtube.com  referral  (none)    NL         Other               1
                                 NZ         iPhone              1
                                 US         Mac                 1
                                            Samsung SM-T827V    1
                                            iPhone              3
Length: 5754, dtype: int64

In [19]:
combs = sliced.groupby(['source', 'medium', 'campaign', 'ipcountry', 'device_family']).size().reset_index().rename(columns={0: 'length'})
combs.head()

,source,medium,campaign,ipcountry,device_family,length
0,(direct),(none),(none),AE,Other,1
1,(direct),(none),(none),AE,iPhone,1
2,(direct),(none),(none),AF,Samsung SM-G950F,2
3,(direct),(none),(none),AT,2203121C,1
4,(direct),(none),(none),AT,Huawei VOG-L29,2


In [20]:
combs['bounce_rate'] = 0
combs['conversion_rate'] = 0
combs['med_duration'] = 0
combs.shape

(5754, 9)

In [21]:
clear_output(wait=True)

rown = 0
pbar = tqdm.tqdm(total=len(combs))
for index, row in combs.iterrows():
#     print(row['source'])
    cuts = sliced[(sliced.source == row.source) & (sliced.medium == row.medium) & 
                  (sliced.campaign == row.campaign) & (sliced.ipcountry == row.ipcountry) &
                  (sliced.device_family == row.device_family)]
    
    bounc = len(cuts[cuts.status_name == 'Bounce'])
    atb = len(cuts[cuts.add_to_basket_count >= 1])
    le = row['length']
#     print(rown, bounc, atb, le)
    
    if bounc != 0:
        try:
            combs.iloc[rown, combs.columns.get_loc('bounce_rate')] = bounc/le
        except ZeroDivisionError:
            combs.iloc[rown, combs.columns.get_loc('bounce_rate')] = 0
    else:
        combs.iloc[rown, combs.columns.get_loc('bounce_rate')] = 0
    
    if atb != 0:
        try:
            combs.iloc[rown, combs.columns.get_loc('conversion_rate')] = atb/le
        except ZeroDivisionError:
            combs.iloc[rown, combs.columns.get_loc('conversion_rate')] = 0
    else:
        combs.iloc[rown, combs.columns.get_loc('conversion_rate')] = 0

    if cuts.duration.sum() != 0:   
        try:
            combs.iloc[rown, combs.columns.get_loc('med_duration')] = cuts.duration.median()
        except ZeroDivisionError:
            combs.iloc[rown, combs.columns.get_loc('med_duration')] = 0
    else:
        combs.iloc[rown, combs.columns.get_loc('med_duration')] = 0
    
    pbar.update(1)
    rown += 1

pbar.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5754/5754 [01:00<00:00, 95.47it/s]


In [22]:
combs.head()

,source,medium,campaign,ipcountry,device_family,length,bounce_rate,conversion_rate,med_duration
0,(direct),(none),(none),AE,Other,1,1.0,0.0,17.170
1,(direct),(none),(none),AE,iPhone,1,0.0,0.0,77.412
2,(direct),(none),(none),AF,Samsung SM-G950F,2,1.0,0.0,135.370
3,(direct),(none),(none),AT,2203121C,1,0.0,0.0,141.854
4,(direct),(none),(none),AT,Huawei VOG-L29,2,0.0,0.0,194.603


In [ ]:
# combs.to_csv('combs.csv')

In [23]:
combs = combs[combs.length >= 10].reset_index(drop=True)
combs.head()

,source,medium,campaign,ipcountry,device_family,length,bounce_rate,conversion_rate,med_duration
6,(direct),(none),(none),AT,Other,15,0.266667,0.000000,108.9300
9,(direct),(none),(none),AT,iPhone,30,0.233333,0.066667,156.1095
20,(direct),(none),(none),BE,iPhone,76,0.421053,0.026316,27.1455
24,(direct),(none),(none),CA,iPhone,15,0.733333,0.000000,8.3100
32,(direct),(none),(none),CH,iPhone,18,0.333333,0.000000,145.2065


In [54]:
(combs.source.nunique())

24

In [69]:
combs.groupby('source').size()

source
(direct)             31
IGShopping           10
facebook.com         19
google.com           21
instagram.com        14
iwearundress.com     35
meta                155
dtype: int64

In [74]:
norms = sliced.groupby(['source']).size().reset_index().rename(columns={0: 'length'})
print(len(norms))
norms = norms[norms['source'].isin(combs.source.unique().tolist())].reset_index(drop=True)
print(len(norms))

93
7


In [75]:
norms['bounce_rate'] = 0
norms['conversion_rate'] = 0
norms['med_duration'] = 0
norms.shape

(7, 5)

In [76]:
clear_output(wait=True)

rown = 0
pbar = tqdm.tqdm(total=len(norms))
for index, row in norms.iterrows():
#     print(row['source'])
    cuts = sliced[(sliced.source == row.source)]
    
    bounc = len(cuts[cuts.status_name == 'Bounce'])
    atb = len(cuts[cuts.add_to_basket_count >= 1])
    le = row['length']
#     print(rown, bounc, atb, le)
    
    if bounc != 0:
        try:
            norms.iloc[rown, norms.columns.get_loc('bounce_rate')] = bounc/le
        except ZeroDivisionError:
            norms.iloc[rown, norms.columns.get_loc('bounce_rate')] = 0
    else:
        norms.iloc[rown, norms.columns.get_loc('bounce_rate')] = 0
    
    if atb != 0:
        try:
            norms.iloc[rown, norms.columns.get_loc('conversion_rate')] = atb/le
        except ZeroDivisionError:
            norms.iloc[rown, norms.columns.get_loc('conversion_rate')] = 0
    else:
        norms.iloc[rown, norms.columns.get_loc('conversion_rate')] = 0

    if cuts.duration.sum() != 0:   
        try:
            norms.iloc[rown, norms.columns.get_loc('med_duration')] = cuts.duration.median()
        except ZeroDivisionError:
            norms.iloc[rown, norms.columns.get_loc('med_duration')] = 0
    else:
        norms.iloc[rown, norms.columns.get_loc('med_duration')] = 0
    
    pbar.update(1)
    rown += 1

pbar.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 225.31it/s]


In [77]:
norms.head()

,source,length,bounce_rate,conversion_rate,med_duration
0,(direct),2408,0.433970,0.019934,32.8780
1,IGShopping,356,0.221910,0.033708,82.7105
2,facebook.com,1192,0.166946,0.011745,80.7195
3,google.com,1595,0.264577,0.031975,105.4340
4,instagram.com,1030,0.072816,0.010680,101.0330


In [78]:
combs.head()

,source,medium,campaign,ipcountry,device_family,length,bounce_rate,conversion_rate,med_duration
6,(direct),(none),(none),AT,Other,15,0.266667,0.000000,108.9300
9,(direct),(none),(none),AT,iPhone,30,0.233333,0.066667,156.1095
20,(direct),(none),(none),BE,iPhone,76,0.421053,0.026316,27.1455
24,(direct),(none),(none),CA,iPhone,15,0.733333,0.000000,8.3100
32,(direct),(none),(none),CH,iPhone,18,0.333333,0.000000,145.2065


In [79]:
kpi = ['bounce_rate', 'conversion_rate', 'med_duration']

In [88]:
tops = pd.DataFrame()
# norms['target_kpi'] = 0
# norms['b_bounce_rate'] = 0
# norms['b_conversion_rate'] = 0
# norms['b_med_duration'] = 0
# norms['d_bounce_rate'] = 0
# norms['d_conversion_rate'] = 0
# norms['d_med_duration'] = 0
for sou in norms.source:
#     print(sou)
    cot = combs[combs.source == sou]
    norm = norms[norms.source == sou]
#     print(type(norm))
#     print(norm['bounce_rate'])
#     print(cot.source)
    for i in range(0, len(kpi)):
        val = kpi[i]
        if val == 'bounce_rate':
            cut = cot.sort_values(val, ascending=True).head(1)
        else:
            cut = cot.sort_values(val, ascending=False).head(1)
        cut['target_kpi'] = val
        cut['kpi_value'] = cut[val]
#         print(val)
        cut['kpi_norma'] = norm[val].iloc[0]
        cut['kpi_delta'] = (cut[val] - norm[val].iloc[0]) / norm[val].iloc[0] * 100
        tops = tops.append(cut)
    

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_157821/1334107378.py:27: FutureWarning:

The frame.append method is deprecated and will be removed from pandas

In [90]:
tops = tops.reset_index(drop=True)

In [91]:
tops

,source,medium,campaign,ipcountry,device_family,length,bounce_rate,conversion_rate,med_duration,target_kpi,kpi_value,kpi_norma,kpi_delta
0,(direct),(none),(none),NL,Generic Smartphone,17,0.000000,0.000000,6.5860,bounce_rate,0.000000,0.433970,-100.000000
1,(direct),(none),(none),ID,Mac,10,0.100000,0.100000,167.0630,conversion_rate,0.100000,0.019934,401.666667
2,(direct),(none),(none),EE,Mac,17,0.176471,0.000000,169.6240,med_duration,169.624000,32.878000,415.919460
3,IGShopping,Social,(none),DK,iPhone,30,0.033333,0.033333,115.9230,bounce_rate,0.033333,0.221910,-84.978903
4,IGShopping,Social,(none),IE,iPhone,16,0.375000,0.125000,55.1745,conversion_rate,0.125000,0.033708,270.833333
5,IGShopping,Social,(none),NL,iPhone,13,0.153846,0.076923,195.5880,med_duration,195.588000,82.710500,136.472999
6,facebook.com,referral,(none),BE,iPhone,22,0.000000,0.000000,99.1290,bounce_rate,0.000000,0.166946,-100.000000
7,facebook.com,referral,(none),US,iPhone,39,0.179487,0.076923,43.3760,conversion_rate,0.076923,0.011745,554.945055
8,facebook.com,referral,(none),SE,iPhone,11,0.090909,0.000000,177.9980,med_duration,177.998000,80.719500,120.514250
9,google.com,referral,(none),DE,iPhone,13,0.000000,0.230769,242.7220,bounce_rate,0.000000,0.264577,-100.000000


In [92]:
tops.to_csv('tops_kpi.csv')